## Setting

In [3]:
#pip install langchain langchain-cohere langchain-groq langchain-community langchain-pinecone python-dotenv

In [ ]:
# Importação de bibliotecas para lidar com variáveis de ambiente
import os
from dotenv import load_dotenv, find_dotenv

# Loaders para carregar arquivos de texto e PDF
from langchain_community.document_loaders import TextLoader

# Ferramentas para dividir documentos em partes menores
from langchain_text_splitters import CharacterTextSplitter

# Geração de embeddings com Cohere
from langchain_cohere import CohereEmbeddings

# Integração com o vetorstore Pinecone
from langchain_pinecone import PineconeVectorStore

# Memória de conversa (não utilizada no trecho atual, mas importada)
from langchain.memory import ConversationBufferMemory

# LLM da Groq (ChatGPT acelerado por hardware especializado)
from langchain_groq import ChatGroq

# Acesso ao hub de chains pré-construídas do LangChain
from langchain import hub

# Chains para combinar documentos e fazer busca com recuperação
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains.retrieval import create_retrieval_chain

# Vectorstore local alternativo (FAISS), não utilizado nesse código
from langchain_community.vectorstores import FAISS

c:\Users\WINDOWS 10\RAG Retrieval-Augmented Generation\RAG-Retrieval-Augmented-Generation-Aprimorando-LLMs-com-Dados-Externos\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Data

Chaves das APIs

In [5]:
# Carrega as variáveis de ambiente do arquivo .env
load_dotenv(find_dotenv())

# Obtém as chaves de API das variáveis de ambiente
COHERE_API_KEY = os.getenv("COHERE_API_KEY")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
GROQ_API_KEY = os.getenv("GROQ_API_KEY")

In [6]:
# Conferindo e pegou as Chaves corretamente
print(f"COHERE_API_KEY: {COHERE_API_KEY}")  # Debug temporário
print(f"PINECONE_API_KEY: {PINECONE_API_KEY}")  # Debug temporário
print(f"GROQ_API_KEY: {GROQ_API_KEY}")  # Debug temporário

COHERE_API_KEY: TOEQPo2gzPWKLP7SUpwAW3svbK3Vwm7tZOkU6FX5
PINECONE_API_KEY: pcsk_65d8mR_TRTfYVcecxyGRyRQPFNARgkRmit2WWY75Q4KVx3YFmmXHA3tiUKWZbsSzYsQSY5
GROQ_API_KEY: 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


Carregando documento

In [7]:
# Carrega o conteúdo do arquivo de texto
loader = TextLoader("../data/mediumblog1.txt")
documents = loader.load()

In [8]:
# Divide o texto em partes menores para facilitar a vetorização
text_splitter = CharacterTextSplitter(separator="\n", chunk_size=1000,chunk_overlap=0)
docs = text_splitter.split_documents(documents)

# Mostra quantas partes foram geradas
print(f"Total de partes criadas: {len(docs)}")


Total de partes criadas: 4


Incorporando agente

In [ ]:
# Instancia os embeddings usando o modelo da Cohere
embeddings = CohereEmbeddings(
    model="embed-english-v3.0",
    cohere_api_key=COHERE_API_KEY
)

In [ ]:
# Nome do índice no Pinecone (deve já existir no console do Pinecone)
index_name = "rag-demo"

# Cria o vetorstore a partir dos documentos, com os embeddings e configurações do Pinecone
vectorstore_from_docs = PineconeVectorStore.from_documents(
    docs,
    embedding=embeddings,
    index_name=index_name,
)

In [11]:
# Teste uma busca semântica:
vectorstore = PineconeVectorStore(index_name=index_name, embedding=embeddings)
print(vectorstore.similarity_search("What is a vector store?"))

[Document(id='05cd1c2a-2322-437e-b72b-1b35a5e74f2f', metadata={'source': 'data/mediumblog1.txt'}, page_content='Title: Vector Database: What is it and Why You Should Know It?\n\nAuthor: Ejiro Onose\nDate: December 22, 2023\n\nIf 2021 was the year of graph databases, 2023 is the year of vector databases â€” Chip Huen.\n\nGenerative AI and Large Language Models (LLMs) have become popular, and a vector database is one of the best tools to handle LLM data. Vector databases provide the ideal infrastructure for managing the complex, high-dimensional data that LLMs produce and rely upon.\n\nIn this article, Iâ€™ll explain what vector databases are, how they work, and introduce some top vector database tools.\n\n What is a Vector?\nIn machine learning (ML), a vector is a collection of numerical values that represents the features of multi-dimensional objects, such as words or images. For example, a vector representing an image might contain values related to pixel intensities and color channel

Criando LLM e Chain

In [12]:
# Inicializa o LLM da Groq com Gemma2
llm = ChatGroq(
    model="Gemma2-9b-It",
    groq_api_key=GROQ_API_KEY,
    temperature=0.1
)                              

In [13]:
# Cria a chain de resposta com base nos documentos
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

retrieval_qa_chat_prompt = hub.pull("langchain-ai/retrieval-qa-chat")
combine_docs_chain = create_stuff_documents_chain(llm, retrieval_qa_chat_prompt)

# Cria o retriever para buscar documentos relevantes
retriever = vectorstore_from_docs.as_retriever()

# Cria a chain de RAG combinando busca + geração
rag_chain = create_retrieval_chain(retriever, combine_docs_chain)

C:\Users\WINDOWS 10\AppData\Local\Temp\ipykernel_4796\2577572416.py:2: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
c:\Users\WINDOWS 10\RAG Retrieval-Augmented Generation\RAG-Retrieval-Augmented-Generation-Aprimorando-LLMs-com-Dados-Externos\.venv\Lib\site-packages\langsmith\client.py:278: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [14]:
# Testanto LLM 
res = rag_chain.invoke({"input":"O que é React? Responda em português."})
print("🔍 Pergunta: O que é React? Responda em português.")
print("💬 Resposta: \n")
print(res['answer'])

🔍 Pergunta: O que é React? Responda em português.
💬 Resposta: 

ReAct é um novo paradigma para combinar raciocínio e ação com modelos de linguagem grandes (LLMs) para resolver tarefas de raciocínio e tomada de decisão em linguagem. 

Em essência, o ReAct incentiva o LLM a gerar tanto traços de raciocínio verbal quanto ações relacionadas a uma tarefa de forma interligada. Isso permite que o modelo realize raciocínio dinâmico para criar, manter e ajustar planos de alto nível para agir ("raciocinar para agir"), ao mesmo tempo em que interage com ambientes externos (como a Wikipedia) para incorporar informações adicionais no raciocínio ("agir para raciocinar").


O texto destaca algumas características importantes do ReAct:

* **Intuitivo e fácil de projetar:** A criação de prompts para o ReAct é simples, pois os anotadores humanos apenas digitam seus pensamentos em linguagem sobre as ações que tomam.
* **Geral e flexível:** Devido ao espaço de pensamento flexível e ao formato de ocorrênci

## Finalização com interação com o usuário

In [15]:
# Função interativa
def fazer_pergunta_rag():
    while True:
        pergunta = input("\n ❓ No que posso ajudar ❓ (ou digite 'sair' para encerrar):\n> ")
        
        if pergunta.lower() in ["sair", "exit", "quit"]:
            print("Encerrando. Até logo!")
            break

        try:
            resposta = rag_chain.invoke({"input":f"{pergunta} Responda em português"})
            print("\n🧠 Resposta:\n")
            print(resposta["answer"])
        except Exception as e:
            print(f"\n⚠️ Erro ao gerar resposta: {e}")


In [16]:
# Execução
if __name__ == "__main__":
    fazer_pergunta_rag()


🧠 Resposta:

ReAct é um framework para tarefas de tomada de decisão e raciocínio sequenciais, baseado em modelos de linguagem grandes. 

Ele funciona assim:

* **Pensamento e Ação:** ReAct permite que um modelo de linguagem "pense" (formulando pensamentos em linguagem natural) e "aja" (executando ações em um ambiente).
* **Trajetórias:** As interações do modelo são representadas como trajetórias, onde cada etapa consiste em um pensamento, uma ação e uma observação resultante.
* **Aprendizado por Exemplo:** ReAct aprende com exemplos de trajetórias pré-definidas, chamadas de "trajetórias densas", onde os pensamentos são explicitamente fornecidos.
* **Flexibilidade:** A estrutura flexível de ReAct permite sua aplicação em diversas tarefas, como perguntas e respostas, verificação de fatos, jogos de texto e navegação na web.

**Vantagens do ReAct:**

* **Intuitivo e Fácil de Projetar:** A criação de prompts para ReAct é simples, pois os humanos apenas digitam os pensamentos em linguagem n